## Prototipo dashaboard voila

imports

In [26]:
from ipywidgets import widgets
import plotly
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd


In [27]:
dataset = pd.read_csv('dataset.csv')
#dataset = dataset.query('State == "Anhui" or  State == "Chongqing"')
dataset.head()

,SNo,ObservationDate,State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020,Anhui,Mainland China,1/22/2020 17:00,1,0,0
1,2,1/22/2020,Beijing,Mainland China,1/22/2020 17:00,14,0,0
2,3,1/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6,0,0
3,4,1/22/2020,Fujian,Mainland China,1/22/2020 17:00,1,0,0
4,5,1/22/2020,Gansu,Mainland China,1/22/2020 17:00,0,0,0


#### exemplo de botão com ipywidgets, criando componentes para interação com o dashboard

In [28]:
dropDown = widgets.Dropdown(
    options=['casa', 'casa2', 'casa3'],
    value='casa',
    description='Opção:',
    disabled=False,
)
output = widgets.Output()
display(dropDown, output)
def observeDropDown(change):
    with output:
        print(change['new'])
    
dropDown.observe(observeDropDown, names='value')

Dropdown(description='Opção:', options=('casa', 'casa2', 'casa3'), value='casa')

Output()

### exemplo de grafico de linha com plotly express
#### para cada país, numero de casos confirmados ao longo do tempo

line_shape = ['linear','spline','vhv','hvh']

In [29]:
dropDown = widgets.Dropdown(
    options=['Confirmed', 'Deaths'],
    value='Confirmed',
    description='Opção:',
    disabled=False,
)
output = widgets.Output()
display(dropDown, output)

def updatePlot(param):
    dataset1 = dataset[['ObservationDate', 'Country',param]]
    fig = px.add_trace(px.line(dataset1, x="ObservationDate", y=param, color="Country", line_group="Country", hover_name="Country",
        line_shape="spline", render_mode="svg"))
    fig.show()

def observeDropDown(change):
    with output:
        return updatePlot(change['new']);
    
dataset1 = dataset[['ObservationDate', 'Country',"Confirmed"]]
fig = px.line(dataset1, x="ObservationDate", y="Confirmed", color="Country", line_group="Country", hover_name="Country",
        line_shape="spline", render_mode="svg")
fig.show()


    
    
dropDown.observe(observeDropDown, names='value')


Dropdown(description='Opção:', options=('Confirmed', 'Deaths'), value='Confirmed')

Output()

### plots sobrepostos
comparativo entre duas series temporais

In [30]:
fig = go.Figure()
dataset1 = dataset[['ObservationDate', 'Country',"Confirmed","Deaths"]]
dataset1 = dataset1.query('Country == "Mainland China"')
fig.add_scatter(x=dataset1.ObservationDate, y=dataset1.Confirmed,name='Confirmed')
fig.add_scatter(x=dataset1.ObservationDate, y=dataset1.Deaths, name='Deaths')
fig.show()



## -------------- predictions -----------------
### Visualizações sugeridas:

Para determinada localidade: Comparativo entre números de casos confirmados e mortes ao longo do tempo.
- insight: o quanto aquela localidade consegue absorver casos confirmados, minimizando mortes.

In [31]:


""" buscarNomeEstadoPorUF """
def buscarNomeEstadoPorUF(uf):
    estados = pd.read_csv('dataset-support/estados.csv')
    try:
        estado = estados.query('uf == "{}"'.format(uf))
        return estado.iloc[0].nome
    except:
        return None
    
def buscarUfEstadoPorNome(nome):
    estados = pd.read_csv('dataset-support/estados.csv')
    try:
        estado = estados.query('nome == "{}"'.format(nome))
        return estado.iloc[0].uf
    except:
        return None
    
def listarNomesEstados():
    estados = pd.read_csv('dataset-support/estados.csv')
    return list(estados[["nome"]].values.flatten())



In [37]:
"""
Usage: python remove_output.py notebook.ipynb [ > without_output.ipynb ]
Modified from remove_output by Minrk
"""
import sys
import io
import os
from IPython.nbformat.current import read, write


def remove_outputs(nb):
    """remove the outputs from a notebook"""
    for ws in nb.worksheets:
        for cell in ws.cells:
            if cell.cell_type == 'code':
                cell.outputs = []

if __name__ == '__main__':
    fname = sys.argv[1]
    with io.open(fname, 'r') as f:
        nb = read(f, 'json')
    remove_outputs(nb)
    base, ext = os.path.splitext(fname)
    new_ipynb = "%s_removed%s" % (base, ext)
    with io.open(new_ipynb, 'w', encoding='utf8') as f:
        write(nb, f, 'json')
    print("wrote {}".format(new_ipynb))

FileNotFoundError: [Errno 2] No such file or directory: '-f'

Para determinada localidade: Comparativo entre o crescimento das curvas em relacao a outros paises

In [36]:
from IPython.display import clear_output

def filtarDataSetPorUf(input):
    dataset = pd.read_csv('datasets/all_diario.csv')
    datasetFiltrado = dataset.query('Country == "{}"'.format(input))
    return datasetFiltrado

def setPlot(dataset):

    fig = go.Figure()
    datasetAux = dataset[['Date', 'Country',"Confirmed","Deaths"]]
    fig.add_scatter(x=datasetAux.Date, y=datasetAux.Confirmed,name='Confirmed')
    fig.add_scatter(x=datasetAux.Date, y=datasetAux.Deaths, name='Deaths')
    fig.layout(title=)
    fig.show()
    return fig
    
dropDown = widgets.Dropdown(
    options=listarNomesEstados(),
    description='Estado:',
    disabled=False,
)
output = widgets.Output()
display(dropDown, output)


def observeDropDown(change):
    with output:
        uf = buscarUfEstadoPorNome(change['new'])
        df = filtarDataSetPorUf(uf)
        nome= buscarNomeEstadoPorUF(uf)
        setPlot(df, )

dropDown.observe(observeDropDown, names='value')
## plot
nome = listarNomesEstados()[0]
uf = buscarUfEstadoPorNome(nome)
df = filtarDataSetPorUf(uf)

fig = setPlot(df)


Dropdown(description='Estado:', options=('Rondônia', 'Acre', 'Amazonas', 'Roraima', 'Pará', 'Amapá', 'Tocantin…

Output()

AttributeError: 'Output' object has no attribute 'worksheets'